# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../output_data/cities.csv"
weather_df = pd.read_csv(file)
weather_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,sola,-13.88,167.55,81.50,80.0,60.0,19.60,VU,1.604180e+09
1,pangnirtung,66.15,-65.71,13.03,79.0,100.0,5.17,CA,1.604180e+09
2,butaritari,3.07,172.79,81.39,75.0,100.0,15.57,KI,1.604180e+09
3,mahebourg,-20.41,57.70,66.20,82.0,75.0,8.05,MU,1.604180e+09
4,atuona,-9.80,-139.03,78.53,75.0,12.0,17.18,PF,1.604180e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
latLongDF = weather_df.loc[:, ["Latitude", "Longitude"]]
coordinatesDF = latLongDF.to_records(index=False)
coordinates = list(coordinatesDF)

In [16]:
figure_layout = {
    'width': '600px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(coordinates, weights=weather_df["Humidity"], 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
trimming_df = weather_df.loc[(weather_df["Temperature"] > 60) & (weather_df["Temperature"] < 75) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] < 20), :]
idealWeather_df = trimming_df.iloc[range(0,10),:]
idealWeather_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
8,saint-philippe,-21.36,55.77,61.11,59.0,0.0,6.93,RE,1.604180e+09
26,salalah,17.02,54.09,73.40,38.0,0.0,5.82,OM,1.604180e+09
27,fukue,32.69,128.84,68.14,58.0,9.0,7.29,JP,1.604180e+09
42,west wendover,40.74,-114.07,60.80,29.0,1.0,4.76,US,1.604180e+09
81,mar del plata,-38.00,-57.56,64.40,55.0,0.0,9.17,AR,1.604180e+09
88,ajdabiya,30.76,20.23,65.91,64.0,0.0,8.95,LY,1.604180e+09
100,fortuna,40.60,-124.16,64.02,63.0,1.0,8.05,US,1.604180e+09
101,port hedland,-20.32,118.57,69.80,94.0,0.0,6.93,AU,1.604180e+09
102,kingaroy,-26.53,151.83,65.55,72.0,0.0,7.00,AU,1.604180e+09
115,kalemie,-5.95,29.19,71.44,66.0,7.0,2.10,CD,1.604180e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in idealWeather_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params).json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        idealWeather_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [26]:
hotel_df = idealWeather_df
hotel_df.head(10)

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,saint-philippe,-21.36,55.77,61.11,59.0,0.0,6.93,RE,1.604180e+09,Les Embruns Du Baril
26,salalah,17.02,54.09,73.40,38.0,0.0,5.82,OM,1.604180e+09,BAITI Hotel Apartments
27,fukue,32.69,128.84,68.14,58.0,9.0,7.29,JP,1.604180e+09,GOTO TSUBAKI HOTEL
42,west wendover,40.74,-114.07,60.80,29.0,1.0,4.76,US,1.604180e+09,Montego Bay Resort
81,mar del plata,-38.00,-57.56,64.40,55.0,0.0,9.17,AR,1.604180e+09,NH Gran Hotel Provincial
88,ajdabiya,30.76,20.23,65.91,64.0,0.0,8.95,LY,1.604180e+09,Almadina Hotel
100,fortuna,40.60,-124.16,64.02,63.0,1.0,8.05,US,1.604180e+09,Best Western Country Inn
101,port hedland,-20.32,118.57,69.80,94.0,0.0,6.93,AU,1.604180e+09,The Esplanade Hotel
102,kingaroy,-26.53,151.83,65.55,72.0,0.0,7.00,AU,1.604180e+09,Kingaroy Holiday Park
115,kalemie,-5.95,29.19,71.44,66.0,7.0,2.10,CD,1.604180e+09,Hotel-Tcham


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [28]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
# Display figure
fig




Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…